In [111]:
import pandas
import numpy as np
import datetime
import matplotlib.pyplot as plt
elo_spd = []
pt_spd = []
def score_elo(hp1,hp2,hp3,hp4,csv): # define a new function that takes in hyperparameter values and returns the percent of games correctly predicted
    data = pandas.read_csv(csv)
    ID = {}
    outcomes = []
    for idx, row in data.iterrows():
        if data.loc[idx,'PTS']>data.loc[idx,'PTS.1']:
            winner = data.loc[idx,'Visitor/Neutral']; loser = data.loc[idx,'Home/Neutral'];
            pd = data.loc[idx,'PTS'] - data.loc[idx,'PTS.1']; hm_i = 0
        else:
            winner = data.loc[idx,'Home/Neutral']; loser = data.loc[idx,'Visitor/Neutral'];
            pd = data.loc[idx,'PTS.1'] - data.loc[idx,'PTS'];hm_i = 1
        game_date = datetime.datetime.strptime(data.loc[idx]["Date"],"%a, %b %d, %Y")
        if winner in ID.keys():
            w_elo = ID[winner][0]
            if (ID[winner][1] - game_date).days == 1: w_date = 1 
            else: w_date = 0
            w_games = ID[winner][2]
        else: 
            w_elo = 1500
            w_date = 0
            w_games = 0
        if loser in ID.keys():
            l_elo = ID[loser][0]
            if (ID[loser][1] - game_date).days == 1: l_date = 1
            else: l_date = 0
            l_games = ID[loser][2]
        else: 
            l_elo = 1500
            l_date = 0
            l_games = 0
        if hm_i == 1: w_elo_g = w_elo+hp3 + hp4*w_date; l_elo_g = l_elo + hp4*l_date
        else: l_elo_g = l_elo +hp3 + hp4*l_date; w_elo_g = w_elo + hp4*l_date;
        dif = w_elo_g - l_elo_g
        if dif>0: pt_spd.append(pd); elo_spd.append(dif)
        else: pt_spd.append(-pd);elo_spd.append(-dif)
        if w_elo_g>l_elo_g: outcomes.append(1)
        if l_elo_g>w_elo_g: outcomes.append(0)
        w_new_elo = (w_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(1-(1/(1+10**((l_elo_g-w_elo_g)/400)))))
        l_new_elo = (l_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(0-(1/(1+10**((w_elo_g-l_elo_g)/400)))))
        ID[winner]=(w_new_elo,game_date,w_games + 1);
        ID[loser]=(l_new_elo,game_date,l_games + 1)
    print (sum(outcomes)/float(len(outcomes)))
score_elo(4,700,5,-.5,'games2015.csv')# run the function on the 2015 data with the given hyperparameters

print (np.polyfit(elo_spd,pt_spd,1))

0.6853658536585366
[ 0.04884846  2.36430226]


In [63]:
print (np.polyfit(elo_spd,pt_spd,1)[1])

2.36430226426


In [108]:
elo_pred= [elo_spds*np.polyfit(elo_spd,pt_spd,1)[0] + np.polyfit(elo_spd,pt_spd,1)[1] for elo_spds in elo_spd]
sub = [(elo_preds - pt_spds) for elo_preds,pt_spds in zip(elo_pred,pt_spd)]
se= [pow(np.float64((elo_preds - pt_spds)).item(),2) for elo_preds,pt_spds in zip(elo_pred,pt_spd)]
print (sum(se)/len(se))
all = list(zip(elo_pred,pt_spd))
print (all)


61.577890637767055
[(10.430402636385047, 18), (10.635566483862707, 17), (10.635566483862707, 1), (10.635566483862707, 16), (10.635566483862707, 2), (10.635566483862707, 10), (10.635566483862707, 12), (10.635566483862707, 4), (10.635566483862707, 12), (10.430402636385047, 24), (10.758123567544899, 20), (10.635566483862707, 17), (10.430402636385047, 18), (10.635566483862707, 7), (10.552959720067241, 11), (10.296422602625729, 5), (10.708935555431566, 18), (10.752459769182442, 3), (10.667453513943082, 6), (10.443563467132689, 7), (10.220236550545421, 6), (10.391759554689854, 8), (10.728603088666327, 7), (10.694868529313821, 12), (10.728187194177865, 5), (10.396116114860778, 9), (10.539759637151381, 10), (10.543084609235072, 2), (10.491082065733703, 12), (11.08377340958315, 23), (10.744943438662544, 14), (10.467450344406874, 1), (10.40312245072267, 5), (10.366706604379546, 11), (10.717763719099104, 13), (10.740589747918246, 18), (10.224504639364302, 27), (10.559912632183565, 11), (10.267224

In [95]:
print (pt_spd)

[6, 11, 7, 9, 11, 33, 9, 1, 2, 22, 22, 2, 13, 14, 17, 10, 7, 3, 11, 28, 2, 8, 13, 14, 8, 17, 10, 5, 13, 22, 13, 11, 4, 9, 11, 14, 19, 19, 10, 10, 8, 8, 9, 8, 9, 11, 3, 6, 11, 17, 2, 12, 11, 5, 18, 8, 3, 31, 6, 8, 1, 2, 28, 7, 12, 22, 8, 11, 4, 1, 14, 7, 20, 28, 11, 16, 16, 8, 15, 17, 9, 7, 3, 12, 25, 3, 28, 3, 15, 10, 5, 7, 16, 28, 23, 13, 12, 41, 28, 2, 4, 7, 12, 1, 18, 14, 14, 3, 19, 23, 25, 14, 4, 6, 23, 3, 13, 9, 20, 9, 10, 19, 7, 11, 26, 15, 12, 8, 14, 16, 22, 9, 23, 6, 6, 4, 9, 10, 1, 4, 3, 3, 5, 10, 8, 1, 4, 2, 6, 6, 18, 6, 13, 13, 8, 8, 14, 15, 14, 5, 7, 15, 14, 13, 14, 2, 6, 13, 17, 15, 7, 7, 2, 5, 2, 14, 10, 7, 7, 19, 5, 7, 6, 19, 7, 5, 7, 8, 28, 22, 3, 16, 16, 1, 2, 26, 8, 4, 11, 24, 14, 8, 14, 10, 11, 10, 1, 3, 35, 10, 31, 13, 13, 3, 1, 15, 10, 14, 14, 17, 19, 13, 7, 3, 8, 8, 21, 10, 25, 18, 14, 10, 13, 16, 33, 2, 16, 16, 6, 6, 10, 19, 7, 12, 9, 16, 8, 19, 2, 19, 2, 27, 18, 28, 11, 27, 7, 4, 31, 7, 17, 17, 2, 16, 11, 14, 5, 2, 18, 2, 7, 7, 14, 9, 1, 16, 6, 9, 18, 9, 2, 1, 3